## Converting labels from png to nii file


### Overview

This is the first step for data preparation

Input: ground truth labels in `.png` format

Output: labels in `.nii` format, indexed by patient id

In [8]:
import numpy as np

import numpy as np
import SimpleITK as sitk


def read_nii_bysitk(input_fid, peel_info = False):
    """ read nii to numpy through simpleitk
        peelinfo: taking direction, origin, spacing and metadata out
    """
    img_obj = sitk.ReadImage(input_fid)
    img_np = sitk.GetArrayFromImage(img_obj)
    if peel_info:
        info_obj = {
                "spacing": img_obj.GetSpacing(),
                "origin": img_obj.GetOrigin(),
                "direction": img_obj.GetDirection(),
                "array_size": img_np.shape
                }
        return img_np, info_obj
    else:
        return img_np

def convert_to_sitk(input_mat, peeled_info):
    """
    write a numpy array to sitk image object with essential meta-data
    """
    nii_obj = sitk.GetImageFromArray(input_mat)
    if peeled_info:
        nii_obj.SetSpacing(  peeled_info["spacing"] )
        nii_obj.SetOrigin(   peeled_info["origin"] )
        nii_obj.SetDirection(peeled_info["direction"] )
    return nii_obj

def np2itk(img, ref_obj):
    """
    img: numpy array
    ref_obj: reference sitk object for copying information from
    """
    itk_obj = sitk.GetImageFromArray(img)
    itk_obj.SetSpacing( ref_obj.GetSpacing() )
    itk_obj.SetOrigin( ref_obj.GetOrigin()  )
    itk_obj.SetDirection( ref_obj.GetDirection()  )
    return itk_obj

In [7]:
%reset
%load_ext autoreload
%autoreload 2
import os
import glob

import numpy as np
import PIL
import matplotlib.pyplot as plt
import SimpleITK as sitk
import sys
sys.path.insert(0, '../../dataloaders/')
import niftiio as nio

Nothing done.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
example = "./MR/1/T2SPIR/Ground/IMG-0002-00001.png" # example of ground-truth file name. 

In [10]:
### search for scan ids
ids = os.listdir("/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR")
OUT_DIR = '/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR'

In [11]:
ids

['3',
 '36',
 '31',
 '22',
 '10',
 '32',
 '38',
 '21',
 '19',
 '13',
 '5',
 '37',
 '2',
 '8',
 '20',
 '39',
 '33',
 '34',
 '15',
 '1']

In [12]:
#### Write them to nii files for the ease of loading in future
for curr_id in ids:
    pngs = glob.glob(f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T2SPIR/Ground/*.png')
    pngs = sorted(pngs, key = lambda x: int(os.path.basename(x).split("-")[-1].split(".png")[0]))
    buffer = []

    for fid in pngs:
        buffer.append(PIL.Image.open(fid))

    vol = np.stack(buffer, axis = 0)
    # flip correction
    vol = np.flip(vol, axis = 1).copy()
    # remap values
    for new_val, old_val in enumerate(sorted(np.unique(vol))):
        vol[vol == old_val] = new_val

    # get reference    
    ref_img = f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T2SPIR/DICOM_anon/IMG-{curr_id}.nii.gz'
    img_o = sitk.ReadImage(ref_img)
    vol_o = nio.np2itk(img=vol, ref_obj=img_o)
    sitk.WriteImage(vol_o, f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T2SPIR/Ground/{curr_id}_seg.nii.gz')
    print(f'image with id {curr_id} has been saved!')

image with id 3 has been saved!
image with id 36 has been saved!
image with id 31 has been saved!
image with id 22 has been saved!
image with id 10 has been saved!
image with id 32 has been saved!
image with id 38 has been saved!
image with id 21 has been saved!
image with id 19 has been saved!
image with id 13 has been saved!
image with id 5 has been saved!
image with id 37 has been saved!
image with id 2 has been saved!
image with id 8 has been saved!
image with id 20 has been saved!
image with id 39 has been saved!
image with id 33 has been saved!
image with id 34 has been saved!
image with id 15 has been saved!
image with id 1 has been saved!


In [15]:
#### Write them to nii files for the ease of loading in future
for curr_id in ids:
    pngs = glob.glob(f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T1DUAL/Ground/*.png')
    pngs = sorted(pngs, key = lambda x: int(os.path.basename(x).split("-")[-1].split(".png")[0]))
    buffer = []

    for fid in pngs:
        buffer.append(PIL.Image.open(fid))

    vol = np.stack(buffer, axis = 0)
    # flip correction
    vol = np.flip(vol, axis = 1).copy()
    # remap values
    for new_val, old_val in enumerate(sorted(np.unique(vol))):
        vol[vol == old_val] = new_val

    # get reference    
    ref_img = f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T1DUAL/DICOM_anon/InPhase/IMG-{curr_id}.nii.gz'
    img_o = sitk.ReadImage(ref_img)
    vol_o = nio.np2itk(img=vol, ref_obj=img_o)
    sitk.WriteImage(vol_o, f'/data_local/mbhosale/CHAOS/CHAOS_Train_Sets/Train_Sets/MR/{curr_id}/T1DUAL/Ground/{curr_id}_seg.nii.gz')
    print(f'image with id {curr_id} has been saved!')

image with id 3 has been saved!
image with id 36 has been saved!
image with id 31 has been saved!
image with id 22 has been saved!
image with id 10 has been saved!
image with id 32 has been saved!
image with id 38 has been saved!
image with id 21 has been saved!
image with id 19 has been saved!
image with id 13 has been saved!
image with id 5 has been saved!
image with id 37 has been saved!
image with id 2 has been saved!
image with id 8 has been saved!
image with id 20 has been saved!
image with id 39 has been saved!
image with id 33 has been saved!
image with id 34 has been saved!
image with id 15 has been saved!
image with id 1 has been saved!


In [ ]:
# TODO We need to save the images if we are using T2 separately, becuase it uses reference image while saving the labels, although ground labels for both T1 and T2 are same.